In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [31]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, padding='post', truncating='post', maxlen=40)

X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences, padding='post', truncating='post', maxlen=40)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, padding='post', truncating='post', maxlen=40)

In [32]:
len(word_index)

23275

In [25]:
from tensorflow import keras
from tensorflow.keras import layers

In [26]:
embedding_dim = 64
vocab_size = len(word_index) + 1

model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1489664   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,601,153
Trainable params: 1,601,153
Non-

In [28]:
from tensorflow.keras import callbacks, metrics

In [29]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    restore_best_weights=True
)

In [39]:
history = model.fit(
    tf.constant(X_train_padded), 
    tf.constant(y_train), 
    validation_data=(tf.constant(X_val_padded), tf.constant(y_val)),
    batch_size=32,
    epochs=30,
    callbacks=[early_stopping]
)

Epoch 1/30
513/513 [==============================] - 45s 89ms/step - loss: 0.2463 - accuracy: 0.9043 - precision: 0.9067 - recall: 0.9013 - val_loss: 0.4174 - val_accuracy: 0.8103 - val_precision: 0.8939 - val_recall: 0.7042
Epoch 2/30
513/513 [==============================] - 46s 91ms/step - loss: 0.1471 - accuracy: 0.9487 - precision: 0.9519 - recall: 0.9452 - val_loss: 0.4358 - val_accuracy: 0.8322 - val_precision: 0.8501 - val_recall: 0.8066
Epoch 3/30
513/513 [==============================] - 46s 90ms/step - loss: 0.0865 - accuracy: 0.9745 - precision: 0.9758 - recall: 0.9732 - val_loss: 0.4997 - val_accuracy: 0.8271 - val_precision: 0.8419 - val_recall: 0.8055
Epoch 4/30
513/513 [==============================] - 46s 90ms/step - loss: 0.0530 - accuracy: 0.9857 - precision: 0.9865 - recall: 0.9849 - val_loss: 0.6246 - val_accuracy: 0.8234 - val_precision: 0.8498 - val_recall: 0.7856
Epoch 5/30
513/513 [==============================] - 49s 96ms/step - loss: 0.0319 - accuracy: 0